In [21]:
from pyspark.sql.functions import (
    when, coalesce, to_date, initcap, trim, col, lit,
    lower, upper, current_timestamp, create_map, expr,
    date_format, round, try_to_timestamp,
    explode, sequence, year, month, dayofmonth, current_date, 
    quarter, concat
)

from pyspark.sql.types import (
    LongType, StringType, IntegerType, DecimalType, TimestampType
)

from datetime import datetime
from itertools import chain

StatementMeta(, 6e8c9473-a207-462c-8da9-82f7e9b09cb6, 23, Finished, Available, Finished)

In [22]:
catalog_name = "Bronze"

StatementMeta(, 108f5b91-7a12-4e32-b566-25530d35b068, 24, Finished, Available, Finished)

# Transformaciones, Validaciones y Guardado, en la Tabla Silver.clientes

In [15]:
email_pattern = "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$"

# 1. Leer tabla Bronze y normalizar nombres de columnas
df = (
    spark.table(f"{catalog_name}.clientes")
    .filter(to_date("ingestion_ts") == current_date())
)
df = df.toDF(*[c.lower() for c in df.columns])

df = (
    df
    .withColumnRenamed("clienteid", "id_cliente"
    )
    .withColumnRenamed("nombrecompleto", "nombre_completo"
    )
    .withColumn("nombre_completo", 
        initcap(trim(col("nombre_completo")))
    )
    .withColumn("ciudad",
        when(col("ciudad").isNull(), None)
        .otherwise(initcap(trim(col("ciudad"))))
    )
    .withColumn("pais",
        when(col("pais").isNull(), None)
        .otherwise(initcap(trim(col("pais"))))
    )

    .withColumn("email",
        when(col("email").isNull(), None)
        .when(lower(trim(col("email"))) == "null", None)
        .otherwise(lower(trim(col("email"))))
    )
    .withColumn("email",
        when(col("email").rlike(email_pattern), col("email"))
        .otherwise(None)
    )
    .dropDuplicates(["id_cliente"])
    .withColumn("updated_at", current_timestamp())
    .select(
            col("id_cliente").cast(LongType()),
            col("nombre_completo").cast(StringType()),
            col("email").cast(StringType()),
            col("ciudad").cast(StringType()),
            col("pais").cast(StringType()),
            col("updated_at")
        )
)

StatementMeta(, 9f85a39a-2d21-4958-8631-15f355ac5df2, 17, Finished, Available, Finished)

In [ ]:
# 2. Crear una vista temporal para usar en SQL
df.createOrReplaceTempView("clientes_temp")

# 3. Ejecutar MERGE con Spark SQL
spark.sql("""
MERGE INTO Silver.clientes AS destino
USING clientes_temp AS origen
ON destino.id_cliente = origen.id_cliente
WHEN MATCHED AND destino.updated_at < origen.updated_at THEN
    UPDATE SET
        destino.id_cliente      = origen.id_cliente,
        destino.nombre_completo = origen.nombre_completo,
        destino.email           = origen.email,
        destino.ciudad          = origen.ciudad,
        destino.pais            = origen.pais,
        destino.updated_at      = origen.updated_at 
WHEN NOT MATCHED THEN
    INSERT (id_cliente, nombre_completo, email, ciudad, pais, updated_at
    )
    VALUES (origen.id_cliente, origen.nombre_completo, origen.email,
        origen.ciudad, origen.pais, origen.updated_at
    )
""")

# Transformaciones, Validaciones y Guardado, en la Tabla Silver.productos

In [10]:
# 1. Leer tabla Bronze y normalizar nombres de columnas
df = spark.table(f"{catalog_name}.productos")
df = df.toDF(*[c.lower() for c in df.columns])

df = (
    df
    .withColumnRenamed("productoid", "id_producto")
    .withColumnRenamed("nombreproducto", "nombre")
    .withColumn("nombre", initcap(trim(col("nombre"))))
    .withColumn("marca",
        when(col("marca").isNull(), None)
        .otherwise(initcap(trim(col("marca"))))
    )
    .withColumnRenamed("preciolista", "precio")
    .withColumn("precio", expr("try_cast(precio as DOUBLE)"))
    .withColumn("precio",
        when((col("precio").isNull()) | (col("precio") <= 0), None)
        .otherwise(round(col("precio"), 2))
    )
    .dropDuplicates(["id_producto"])
    .withColumn("updated_at", current_timestamp())
    .select(
        col("id_producto").cast(LongType()),
        col("nombre").cast(StringType()),
        col("marca").cast(StringType()),
        col("precio").cast(DecimalType(18, 2)),
        col("updated_at")
    )
)

StatementMeta(, 9f85a39a-2d21-4958-8631-15f355ac5df2, 12, Finished, Available, Finished)

In [ ]:
# 2. Crear una vista temporal para usar en SQL
df.createOrReplaceTempView("productos_temp")

# 3. Ejecutar MERGE con Spark SQL
spark.sql("""
MERGE INTO Silver.productos AS destino
USING productos_temp AS origen
ON destino.id_producto = origen.id_producto
WHEN MATCHED THEN
    UPDATE SET
        destino.nombre     = origen.nombre,
        destino.marca      = origen.marca,
        destino.precio     = origen.precio,
        destino.updated_at = origen.updated_at
WHEN NOT MATCHED THEN
    INSERT (id_producto, nombre, marca, precio, updated_at)
    VALUES (origen.id_producto, origen.nombre, origen.marca, origen.precio, origen.updated_at)
""")


# Transformaciones, Validaciones y Guardado, en la Tabla Silver.Tiendas

In [1]:
# 1. Leer tabla Bronze y normalizar nombres de columnas
df = spark.table(f"{catalog_name}.tiendas")
df = df.toDF(*[c.lower() for c in df.columns])

df = (
    df
    .withColumnRenamed("tiendaid", "id_tienda")
    .withColumnRenamed("nombretienda", "nombre")
    .dropDuplicates(["id_tienda"])
    .withColumn("updated_at", current_timestamp())
    .select(
        col("id_tienda").cast(LongType()),
        initcap(trim(col("nombre"))).alias("nombre"),
        initcap(trim(col("ciudad"))).alias("ciudad"),
        upper(trim(col("pais"))).alias("pais"),
        upper(trim(col("tipo"))).alias("tipo"),
        col("updated_at")
    )
)

StatementMeta(, 6561cd66-461c-4ee8-a4ee-6dea1d5acc03, 3, Finished, Available, Finished)

NameError: name 'catalog_name' is not defined

In [ ]:
# 2. Crear una vista temporal para usar en SQL
df.createOrReplaceTempView("tiendas_temp")

# 3. Ejecutar MERGE con Spark SQL
spark.sql("""
MERGE INTO Silver.tiendas AS destino
USING tiendas_temp AS origen
ON destino.id_tienda = origen.id_tienda
WHEN MATCHED THEN
    UPDATE SET
        destino.nombre     = origen.nombre,
        destino.ciudad     = origen.ciudad,
        destino.pais       = origen.pais,
        destino.tipo       = origen.tipo,
        destino.updated_at = origen.updated_at
WHEN NOT MATCHED THEN
    INSERT (id_tienda, nombre, ciudad, pais, tipo, updated_at)
    VALUES (origen.id_tienda, origen.nombre, origen.ciudad, origen.pais, origen.tipo, origen.updated_at)
""")


# Transformaciones, Validaciones y Guardado, en la Tabla Silver.ventas

In [11]:
numeros_literales = {
    "cero": "0", "uno": "1", "dos": "2", "tres": "3",
    "cuatro": "4", "cinco": "5", "seis": "6", "siete": "7",
    "ocho": "8", "nueve": "9", "diez": "10"
}

mapping_numeros = create_map([lit(x) for x in chain(*numeros_literales.items())])

# 1. Leer tabla Bronze y normalizar nombres de columnas
df = spark.table(f"{catalog_name}.ventas")
df = df.toDF(*[c.lower() for c in df.columns])

# Fecha actual 
fecha_actual = datetime.now().date()

df = (
    df
    .withColumnRenamed("ventaid", "id_venta")
    .withColumnRenamed("productoid", "id_producto")
    .withColumnRenamed("clienteid", "id_cliente")
    .withColumnRenamed("tiendaid", "id_tienda")
    .withColumnRenamed("preciounitario", "precio_unitario")
    .withColumn(
        "cantidad_mapeada",
        coalesce(
            mapping_numeros[lower(trim(col("cantidad")))],
            col("cantidad")
        )
    )
    .withColumn(
        "cantidad",
        when(
            expr("try_cast(cantidad_mapeada as INT)") > 0,
            expr("try_cast(cantidad_mapeada as INT)")
        ).otherwise(None)
    )
    .drop("cantidad_mapeada")
    .withColumn(
        "monto",
        when(
            (col("cantidad").isNotNull()) & (col("precio_unitario").isNotNull()),
            round(col("cantidad") * col("precio_unitario"))
        ).otherwise(None)
    )
    .withColumn(
        "monto",
        when(
            expr("try_cast(monto as DOUBLE)") > 0,
            round(expr("try_cast(monto as DOUBLE)"), 2)
        ).otherwise(None)
    )
    .withColumnRenamed("fechaventa", "fecha_venta")
    .withColumn(
        "fecha_venta",
        expr("""
            coalesce(
                try_to_timestamp(trim(fecha_venta), 'yyyy/MM/dd HH:mm:ss'),
                try_to_timestamp(trim(fecha_venta), 'yyyy-MM-dd HH:mm:ss'),
                try_to_timestamp(trim(fecha_venta), 'dd/MM/yyyy HH:mm:ss'),
                try_to_timestamp(trim(fecha_venta), 'yyyy-MM-dd'),
                try_to_timestamp(trim(fecha_venta), 'yyyy/MM/dd'),
                try_to_timestamp(trim(fecha_venta), 'dd/MM/yyyy')
            )
        """)
    )
    .withColumn(
        "fecha_venta",
        when(
            (col("fecha_venta") > lit(fecha_actual)) | 
            (col("fecha_venta") < lit("2023-12-31")),
            None
        ).otherwise(col("fecha_venta"))
    )
    .withColumn(
        "anio_mes",
        when(
            col("fecha_venta").isNotNull(),
            date_format(col("fecha_venta"), "yyyy-MM")
        )
    )
    .dropDuplicates(["id_venta"])
    .withColumn("updated_at", current_timestamp())
    .select(
        col("id_venta").cast(LongType()),
        col("id_cliente").cast(LongType()),
        col("id_tienda").cast(LongType()),
        col("id_producto").cast(LongType()),
        col("cantidad").cast(IntegerType()),
        col("monto").cast(DecimalType(18, 2)),
        col("fecha_venta").cast(TimestampType()),
        col("anio_mes").cast(StringType()),
        col("updated_at")
    )
)

StatementMeta(, 6561cd66-461c-4ee8-a4ee-6dea1d5acc03, 13, Finished, Available, Finished)

In [ ]:
# 2. Crear una vista temporal para usar en SQL
df.createOrReplaceTempView("ventas_temp")

# 3. Ejecutar MERGE con Spark SQL

spark.sql("""
MERGE INTO Silver.ventas AS destino
USING ventas_temp AS origen
ON destino.id_venta = origen.id_venta
WHEN MATCHED THEN
    UPDATE SET
        destino.id_cliente   = origen.id_cliente,
        destino.id_tienda    = origen.id_tienda,
        destino.id_producto  = origen.id_producto,
        destino.cantidad     = origen.cantidad,
        destino.monto        = origen.monto,
        destino.fecha_venta  = origen.fecha_venta,
        destino.anio_mes     = origen.anio_mes,
        destino.updated_at   = origen.updated_at
WHEN NOT MATCHED THEN
    INSERT (id_venta, id_cliente, id_tienda, id_producto, cantidad, monto, fecha_venta, anio_mes, updated_at)
    VALUES (origen.id_venta, origen.id_cliente, origen.id_tienda, origen.id_producto,
            origen.cantidad, origen.monto, origen.fecha_venta, origen.anio_mes, origen.updated_at)
""")


In [24]:
start_date = "2024-01-01"
end_date   = "2024-12-31"

# Verificar si la tabla ya existe en el catálogo Silver
if not spark.catalog.tableExists("Silver.tiempo"):
    df = (
        spark.range(1)
        .withColumn("fecha", explode(sequence(
            expr(f"to_date('{start_date}')"),
            expr(f"to_date('{end_date}')"),
            expr("interval 1 day")
        )))
        .select("fecha")
        .withColumn("anio", year(col("fecha")))
        .withColumn("mes", month(col("fecha")))
        .withColumn("dia", dayofmonth(col("fecha")))
        .withColumn("anio_mes", date_format(col("fecha"), "yyyyMM"))
        .withColumn("anio_trimestre",concat(col("anio"),lit("T"),quarter(col("fecha"))))
        .withColumn("sk_tiempo", date_format(col("fecha"), "yyyyMMdd").cast("int"))
        .withColumn("updated_at", current_timestamp())
        .select("sk_tiempo", "fecha", "anio", "mes", "dia", "anio_mes", "anio_trimestre", "updated_at")
    )

    df.write.format("delta").mode("overwrite").saveAsTable("Silver.tiempo")
else:
    print("⚡ La tabla Silver.tiempo ya existe, no se ejecuta la función.")


StatementMeta(, 6e8c9473-a207-462c-8da9-82f7e9b09cb6, 26, Finished, Available, Finished)